# Intial tests for NEXT real-time analysis

In [1]:
import numpy as np
import socket
import struct
import time

from pynq import Xlnk
from pynq import Overlay
from pynq.mmio import MMIO

In [2]:
# Load the Overlay
overlay = Overlay("/home/xilinx/jupyter_notebooks/nextrta/evtsender.bit")

# Initialize Xlnk
xlnk = Xlnk()

# Get the DMA object for transfers
dma = overlay.axi_dma_0

### Receive many events over ethernet

In [8]:
def open_tcp_listener(ip="0.0.0.0", port=12345):
    """Open a socket, bind, listen, accept once. Return the connected `conn`."""
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((ip, port))
    s.listen(1)
    print(f"Listening on {ip}:{port}...")
    conn, addr = s.accept()
    print(f"Accepted connection from {addr}")
    return conn

def receive_one_event(conn):
    """Receive one event from an already-connected socket."""
    # 1) Read 4 bytes => eventSize
    size_data = conn.recv(4)
    if not size_data or len(size_data) < 4:
        return None  # No data => end of stream or error

    event_size = struct.unpack("<I", size_data)[0]
    event_data = bytearray(size_data)

    # 2) Read event_size - 4 more bytes
    remaining = event_size - 4
    while remaining > 0:
        chunk = conn.recv(remaining)
        if not chunk:
            break
        event_data.extend(chunk)
        remaining -= len(chunk)
    
    if len(event_data) < event_size:
        print("Incomplete event or connection closed unexpectedly.")
        return None

    print(f"Received one event of size {len(event_data)}")
    return event_data

In [24]:
conn = open_tcp_listener("0.0.0.0", 12345)

Listening on 0.0.0.0:12345...
Accepted connection from ('192.168.2.1', 55854)


In [25]:
start_time = time.time()
n_events = 64
total_bytes = 0

# Pre-allocate once for 10 MB max event size
MAX_EVENT_SIZE = 10 * 1024 * 1024  # 10 MB
cma_buf_in = xlnk.cma_array(shape=(MAX_EVENT_SIZE//4,), dtype=np.uint32)
cma_buf_out = xlnk.cma_array(shape=(MAX_EVENT_SIZE//4,), dtype=np.uint32)

for i in range(n_events):
    event_data = receive_one_event(conn)
    if not event_data:
        print(f"Stopped at event {i}")
        break

    # Convert to np array
    event_size = len(event_data)
    arr32 = np.frombuffer(event_data, dtype=np.uint32)
    num_words = arr32.size
    
    # Copy the data to the input buffer
    cma_buf_in[:num_words] = arr32  

    # DMA
    dma.sendchannel.transfer(cma_buf_in)
    dma.recvchannel.transfer(cma_buf_out)
    dma.sendchannel.wait()
    dma.recvchannel.wait()

    # Check correctness
#     final_output = bytearray(event_size)
#     np.copyto(np.frombuffer(final_output, dtype=np.uint32), cma_buf_out[:num_words])
#     if final_output != event_data:
#         print(f"Mismatch in event {i}")

    total_bytes += event_size

end_time = time.time()
conn.close()

elapsed = end_time - start_time
mb_transferred = total_bytes / 1e6
print(f"Received + DMA {n_events} events in {elapsed:.3f} s.")
print(f"Rate: {n_events/elapsed:.2f} events/s, {mb_transferred/elapsed:.2f} MB/s")

Received one event of size 8298312
Received one event of size 8292512
Received one event of size 8302056
Received one event of size 8306940
Received one event of size 8321504
Received one event of size 8271348
Received one event of size 8260144
Received one event of size 8260804
Received one event of size 8445500
Received one event of size 8331520
Received one event of size 8192844
Received one event of size 8318416
Received one event of size 8219288
Received one event of size 8206484
Received one event of size 8345064
Received one event of size 8328732
Received one event of size 8175308
Received one event of size 8253512
Received one event of size 8325980
Received one event of size 8391888
Received one event of size 8269020
Received one event of size 8233424
Received one event of size 8310152
Received one event of size 8235040
Received one event of size 8282760
Received one event of size 8299308
Received one event of size 8310412
Received one event of size 8425116
Received one event o

### Receive ONE EVENT over ethernet

In [ ]:
def receive_event_from_network(ip="0.0.0.0", port=12345):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.bind((ip, port))
    s.listen(1)
    conn, addr = s.accept()
    print(f"Accepted connection from {addr}")

    # Read 4 bytes => eventSize
    size_data = conn.recv(4)
    if len(size_data) < 4:
        print("No data or incomplete header")
        return None

    event_size = struct.unpack("<I", size_data)[0]
    event_data = bytearray(size_data)

    # Now read the rest
    remaining = event_size - 4
    while remaining > 0:
        chunk = conn.recv(remaining)
        if not chunk:
            print("Connection closed prematurely.")
            break
        event_data.extend(chunk)
        remaining -= len(chunk)

    print(f"Received event of size {len(event_data)} bytes.")
    return event_data

In [ ]:
event_data = receive_event_from_network()

In [ ]:
event_size = len(event_data)

arr32 = np.frombuffer(event_data, dtype=np.uint32)
num_words = event_size // 4

# CMA buffers
event_data_32 = xlnk.cma_array(shape=(num_words,), dtype=np.uint32)
np.copyto(event_data_32, arr32)
output_data_32 = xlnk.cma_array(shape=(event_size//4,), dtype='uint32')

dma.sendchannel.transfer(event_data_32)
dma.recvchannel.transfer(output_data_32)
dma.sendchannel.wait()
dma.recvchannel.wait()

# Reconstruct final bytes (if needed)
final_output = bytearray(event_size)
np.copyto(np.frombuffer(final_output, dtype=np.uint32), output_data_32)

In [ ]:
event_data == final_output
#event_data[0] = final_output[0]

### DMA test

In [ ]:
# Allocate buffers
input_buffer = xlnk.cma_array(shape=(1024,), dtype=np.uint32)
output_buffer = xlnk.cma_array(shape=(1024,), dtype=np.uint32)

In [ ]:
hex(input_buffer.physical_address)

In [ ]:
hex(output_buffer.physical_address)

In [ ]:
# Fill input buffer with test pattern
for i in range(1024):
    input_buffer[i] = i

# Start the DMA transfer
dma.sendchannel.transfer(input_buffer)
dma.recvchannel.transfer(output_buffer)

# Wait for completion
dma.sendchannel.wait()
dma.recvchannel.wait()

# Check results
print("i ibuf obuf")
for i in range(100):
    #print(i, hex(input_buffer[i]), hex(output_buffer[i]))
    print(i, input_buffer[i], output_buffer[i])